# Frequent itemset mining

- Student ID: 21127621
- Student name: Âu Dương Khang

**How to do your homework**


You will work directly on this notebook; the word `TODO` indicate the parts you need to do.

You can discuss ideas with classmates as well as finding information from the internet, book, etc...; but *this homework must be your*.

**How to submit your homework**

Before submitting, rerun the notebook (`Kernel` ->` Restart & Run All`).

Then create a folder named `ID` (for example, if your ID is 1234567, then name the folder `1234567`) Copy file notebook to this folder, compress and submit it on moodle.

**Contents:**

- Frequent itemset mining.

# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example: 

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [194]:
from collections import defaultdict

### Read data
First we have to read data from database

In [195]:

def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) #* Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        tid=1;
        for line in f:
            itemset=set(map(int,line.split())) #* A python set is a native way for storing an itemset.
            for item in itemset:  
                s[item]+=1     #* Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1
            
    return data, s

### Tree Projection

**Question 0: I gave you pseudo code of Apriori algorithm above but we implement Tree Projection. Tell me the differences of two algorithms.**


**Answer:**

*Apriori Algorithm*:

- It’s an array-based algorithm.
- It uses a breadth-first search strategy.
- It utilizes a level-wise approach where it generates patterns containing 1 item, then 2 items, then 3 items, and so on.
- It uses a Join and Prune technique.
- It generates candidate itemsets and tests if they are frequent.
- It discovers the itemset which is frequent, then all of its subsets must also be frequent.

*Tree Projection Algorithm*:

- It uses a cost-effective and efficient ‘divide and conquer’ strategy.
- It bypasses candidate generation and builds a tree.
- It uses pattern fragment growth to mine the frequent patterns from large database.
- It uses a depth-first search strategy.
- It generates a compressed database of frequent patterns.

*Here’s a table that summarizes the key differences between the Apriori and Tree Projection algorithms*:
| Aspect| Apriori | Tree Projection |
| --- | --- | --- |
| Data Structure	 | Array-based | Tree-based |
| Search Strategy| Breadth-first search | Depth-first search |
| Approach| Level-wise, generates patterns containing 1 item, then 2 items, etc | Uses pattern fragment growth |
| Technique| Join and Prune	 | Bypasses candidate generation |
| Efficiency| Can be computationally expensive for large datasets	 | More efficient for large datasets |

In [196]:
def joinset(a,b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space) 

    -------------------
    return 
        ret: itemset generated by joining a and b
    '''
    # a,b same brach -> a_k = b_k
    # Hint: this function will be called in generateSearchSpace method.:
    ret = list(set(a) | set(b))
    return ret
    
class TP:
    def __init__ (self, data=None, s=None, minSup=None):
        self.data=data
        self.s={}
        
        
        for key, support in sorted(s.items(),key= lambda item: item[1]):
            self.s[key]=support
        # Why should we do this, answer it at the markdown below?
            
        self.minSup=minSup
        self.L={}  #Store frequent itemsets mined from database
        self.runAlgorithm()
    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree={}

        search_space={}  
        for item, support in self.s.items():
            search_space[item]={}
            
            search_space[item]['itemset']=[item] 
            ''' 
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset 
            but why we store itemset in data by a python set???? '''
            # TODO: study about python set and its advantages,
            # answer at the markdown below.
            
            search_space[item]['pruned'] = False

            
            search_space[item]['support'] = support
            
            tree[item]={}
            '''
            Why should i store an additional tree (here it called tree)? 
            Answer: This really help in next steps.
            
            Remember that there is always a big gap from theory to practicality
            and implementing this algorithm in python is not as simple as you think.
            ''' 
        
        return tree, search_space
    
    def computeItemsetSupport(self, itemset):
        '''Return support of itemset'''
        # TODO (hint: this is why i use python set in data)
        support = sum(1 for transaction in self.data.values() if set(itemset).issubset(transaction))
        return support
    
        
    def prune(self,k, tree, search_space):
        
        '''
        In this method we will find out which itemset in current search space is frequent 
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets. 
        '''
        # Initialize L[k] if it doesn't exist
        self.L.setdefault(k, [])
        # TODO
        for key in search_space.keys():
            if search_space[key]['support'] < self.minSup:
                tree.pop(key)
                search_space[key]['pruned'] = True
            else:
                search_space[key]['pruned'] = False
                self.L[k].append(search_space[key]['itemset'])
       
    def generateSearchSpace(self,k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function) 
        '''
        items=list(tree.keys())  
        ''' print search_space.keys() you will understand  
         why we need an additional tree, '''
        l=len(items)
        self.prune(k, tree, search_space)
        if l==0: return   #Stop condition
        for i in range(l - 1):
            sub_search_space = {}
            sub_tree = {}
            a = items[i]
            if search_space[a]['pruned']: continue

            for j in range(i + 1, l):
                b = items[j]
                search_space[a][b] = {}
                tree[a][b] = {}
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft).

                # TODO:
                # First create newset using join set
                newset=joinset(search_space[a]['itemset'], search_space[b]['itemset'])
                # Second add newset to search_space
                search_space[a][b]['itemset'] = newset
                search_space[a][b]['pruned'] = False 
                search_space[a][b]['support'] = self.computeItemsetSupport(newset)
                
                sub_search_space[b] = search_space[a][b]
                sub_tree[b] = {}
            #  Generate search_space for k+1-itemset
            self.generateSearchSpace(k + 1, sub_tree, sub_search_space)

    
    def runAlgorithm(self):
        tree,search_space=self.initialize() #generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)
    def miningResults(self):
        return self.L
        

Ok, let's test on a typical dataset `chess`.

In [197]:
transactions, freq= readData('chess.txt')

In [198]:
# Run and print result (better print format)
a=TP(data=transactions,s=freq, minSup=3000)
print(*a.miningResults().items(),sep="\n")

(1, [[48], [56], [66], [34], [62], [7], [36], [60], [40], [29], [52], [58]])
(2, [[48, 52], [48, 58], [56, 29], [56, 52], [56, 58], [66, 60], [66, 29], [66, 52], [66, 58], [40, 34], [34, 29], [34, 52], [34, 58], [60, 62], [40, 62], [29, 62], [52, 62], [58, 62], [60, 7], [40, 7], [29, 7], [52, 7], [58, 7], [36, 60], [40, 36], [36, 29], [36, 52], [58, 36], [40, 60], [60, 29], [60, 52], [58, 60], [40, 29], [40, 52], [40, 58], [52, 29], [58, 29], [58, 52]])
(3, [[48, 58, 52], [56, 52, 29], [56, 58, 29], [56, 58, 52], [66, 60, 29], [66, 60, 52], [66, 60, 58], [66, 52, 29], [66, 58, 29], [66, 52, 58], [40, 34, 29], [40, 34, 52], [40, 34, 58], [34, 52, 29], [34, 58, 29], [34, 52, 58], [60, 29, 62], [60, 62, 52], [58, 60, 62], [40, 29, 62], [40, 52, 62], [40, 58, 62], [52, 29, 62], [58, 29, 62], [58, 52, 62], [40, 60, 7], [60, 29, 7], [60, 52, 7], [58, 60, 7], [40, 29, 7], [40, 52, 7], [40, 58, 7], [52, 29, 7], [58, 29, 7], [58, 52, 7], [40, 36, 60], [36, 29, 60], [36, 60, 52], [58, 36, 60], [

### Answer questions here:


**Question 1. Why should we sort all items in s by it's support and why we have to restored it to new artribute s?**

**Answer:**

There are 2 reason why we should sort all items in s: 

- Efficiency: Sorting items by their support can improve the efficiency of the algorithm. Items with higher support are more likely to appear in frequent itemsets, so considering these items first can allow the algorithm to discover frequent itemsets more quickly.

- Pruning: In the Apriori algorithm, for example, the support property is used to prune the search space. If an itemset does not meet the minimum support threshold, then its supersets will also not meet the minimum support threshold, and they can be pruned from the search space. Sorting items by their support can make this pruning process more efficient.


**Question 2. Python set does not remain elements order so we use a list to extend it easily when create new itemset but why we store itemset in data by a python set????**

**Answer:**

Storing itemsets as sets in Python has a couple of key advantages that can be particularly useful in the context of frequent itemset mining algorithms like Apriori:

1. *Efficient Membership Tests*: Sets in Python are implemented as hash tables, which provide constant time complexity, O(1), for checking if an item exists in the set (membership test). This is particularly beneficial when you need to quickly check if an item is part of an itemset, which is a common operation in frequent itemset mining algorithms.

2. *Uniqueness of Items*: Sets automatically ensure that all elements are unique. This can be useful to avoid duplicate items within an itemset.

3. *Set Operations*: Python sets support operations like union, intersection, difference, and symmetric difference. These operations can be useful when manipulating itemsets, such as when generating candidate itemsets.

However, it's important to note that sets do not maintain the order of elements. If the order of items within the itemsets is important for your specific use case or algorithm, then a different data structure might be more appropriate. 

For example, you might use a list to maintain the order of items when generating candidate itemsets, and a set or dictionary to store the support counts of itemsets for efficient lookup. The specific implementation can vary depending on the requirements of the task and the characteristics of the data.

**Question 3.  After finish implementing the algorithm tell me why should you use this instead of delete item directly from search_space and tree.**

**Answer:**

Certainly, the Apriori algorithm and its implementation details, such as the use of a 'pruned' attribute, have several noteworthy aspects:

1. *Efficient Pruning*: The use of a 'pruned' attribute allows for efficient pruning of the search space. By marking itemsets as 'pruned' instead of deleting them, the algorithm can avoid costly operations associated with deletion and prevent potential errors that could arise from modifying a data structure during iteration.

2. *Preservation of Information*: Marking an itemset as 'pruned' retains information about the itemset within the data structure. This can be useful for debugging, understanding the workings of the algorithm, or even revisiting pruned itemsets if the criteria for pruning change.

3. *Flexibility and Adaptability*: The Apriori algorithm's design allows for flexibility and adaptability. For instance, the use of different data structures (like sets for efficient membership tests and lists for preserving order) enables the algorithm to handle various requirements effectively.

4. *Scalability*: Despite its simplicity, the Apriori algorithm scales well with increasing data size, especially when optimizations like efficient pruning and transaction reduction are used.

5. *Broad Applicability*: The Apriori algorithm has broad applicability in many domains, including market basket analysis, network traffic analysis, and bioinformatics, among others.

In conclusion, the Apriori algorithm's design, including the use of a 'pruned' attribute and the choice of appropriate data structures, contributes to its effectiveness and wide use in frequent itemset mining tasks. It's a testament to the power of clever algorithm design and the thoughtful use of data structures.


# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes). You can download dataset from here: http://ce.sharif.edu/courses/85-86/1/ce925/assignments/files/assignDir2/churn.txt. Write your report and implementation below.

*Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.  
More information about `churn` here: http://ce.sharif.edu/courses/85-86/1/ce925/assignments/files/assignDir4/Churn.pdf)*

**TODO:**

**Import library**

In [199]:
import csv

**Read data from file 'churn.txt' and transforming it into a transactional database**

In [200]:
def read_churn_file(filename):
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        headers = next(reader)
        for row in reader:
            data = {header: value for header, value in zip(headers, row)}
            yield data

def save_to_csv(data, filename):
    with open(filename, 'w', newline='') as csvfile:
        if data:
            writer = csv.DictWriter(csvfile, fieldnames=data[0].keys())
            writer.writeheader()
            writer.writerows(data)
            
data = list(read_churn_file('churn.txt'))
save_to_csv(data, 'churn.csv')

def read_csv_file(filename):
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        matrix = list(reader)
    return matrix

# Use the function
matrix = read_csv_file('churn.csv')

# Split each row into a list of values.
dataset = [row for row in matrix]
dataset = dataset[1:]


def save_to_txt(dataset, filename):
    with open(filename, 'w') as f:
        for row in dataset:
            f.write(' '.join(row) + '\n')

# Use the function
save_to_txt(dataset, 'churn_data.txt')


def read_file(path):
    data={}
    s=defaultdict(lambda: 0)   
    with open(path,'rt') as f:
        tid=1;
        for line in f:
            itemset=set(map(str,line.split())) 
            for item in itemset:  
                s[item]+=1   
            data[tid]= itemset
            tid+=1
            
    return data, s
            

**Test with file churn_data.txt**

In [201]:
transactions, freq= read_file('churn_data.txt')

In [202]:
# Run and print result (better print format)
a=TP(data=transactions,s=freq, minSup=1000)
print(*a.miningResults().items(),sep="\n")

(1, [['3'], ['2'], ['yes'], ['1'], ['415'], ['0'], ['False.'], ['no']])
(2, [['2', 'no'], ['yes', 'no'], ['1', 'False.'], ['1', 'no'], ['0', '415'], ['False.', '415'], ['no', '415'], ['0', 'False.'], ['0', 'no'], ['no', 'False.']])
(3, [['1', 'no', 'False.'], ['0', 'False.', '415'], ['0', 'no', '415'], ['no', 'False.', '415'], ['0', 'no', 'False.']])
(4, [['False.', '415', '0', 'no']])


The frequent itemset mining technique was applied to the `churn` dataset to identify patterns and associations among the data. The dataset was first read from a text file and then saved as a CSV file for easier manipulation. Each row in the dataset was split into a list of values to form a matrix.

The frequent itemsets were then identified using the TP function with a minimum support of 1000. The results are as follows:

- Frequent 1-itemsets: [‘3’], [‘2’], [‘yes’], [‘1’], [‘415’], [‘0’], [‘False.’], [‘no’]
- Frequent 2-itemsets: [‘2’, ‘no’], [‘yes’, ‘no’], [‘1’, ‘False.’], [‘1’, ‘no’], [‘0’, ‘415’], [‘False.’, ‘415’], [‘no’, ‘415’], [‘0’, ‘False.’], [‘0’, ‘no’], [‘no’, ‘False.’]
- Frequent 3-itemsets: [‘1’, ‘no’, ‘False.’], [‘0’, ‘False.’, ‘415’], [‘0’, ‘no’, ‘415’], [‘no’, ‘False.’, ‘415’], [‘0’, ‘no’, ‘False.’]
- Frequent 4-itemsets: [‘False.’, ‘415’, ‘0’, ‘no’]

These frequent itemsets represent common combinations of values in the dataset. For example, the 4-itemset [‘False.’, ‘415’, ‘0’, ‘no’] indicates that these four values frequently appear together in the dataset.

The code provided reads the `churn` dataset, saves it as a CSV file, reads the CSV file into a matrix, and then saves the matrix as a text file. It then reads the text file and applies the frequent itemset mining technique to identify frequent itemsets.

This analysis can be useful for understanding patterns in customer churn and could potentially inform strategies for customer retention. For example, if a particular combination of factors is frequently associated with churn, interventions could be targeted towards customers with these characteristics to prevent them from churning.

# 4. References

http://ce.sharif.edu/courses/85-86/1/ce925/assignments/files/assignDir2/ProjectDefinition1.pdf

https://cs.wmich.edu/~alfuqaha/summer14/cs6530/lectures/AssociationAnalysis-Part1.pdf

Feel free to send questions to my email address: nnduc@fit.hcmus.edu.vn
